# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import (weather_api_key, g_key)
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../Output/city_weather_data.csv")
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,New,6.6518,-10.5932,77.00,94,20,3.04,LR,1613005169
1,Upernavik,72.7868,-56.1549,-2.63,91,100,13.04,GL,1613005169
2,Isil'kul',54.9109,71.2728,-27.15,84,1,3.91,RU,1613005169
3,Ribeira,42.7461,-8.4439,50.00,100,75,9.22,ES,1613005169
4,Souillac,-20.5167,57.5167,78.80,73,40,9.22,MU,1613005169


In [12]:
weather_df[['Max Temp','Humidity','Cloudiness','Wind Speed']].agg(['mean','median','min','max','std'])

,Max Temp,Humidity,Cloudiness,Wind Speed
mean,48.117413,72.807339,54.207339,7.425853
median,57.990000,78.000000,75.000000,5.750000
min,-32.800000,13.000000,0.000000,0.130000
max,90.590000,100.000000,100.000000,37.920000
std,31.044146,20.455330,41.810927,5.753517


### Humidity Heatmap

In [4]:
locations = weather_df[['Lat','Lng']]
humidity = weather_df['Humidity']

figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heatmap = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False)
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Personally I love hot weather:

    -Temperature between between 70 and 85 F.    
    
    -Humidity is less than 70%.
    
    -Cloudiness is 0.   


* Drop any rows will null values.

In [5]:
ideal_locations = weather_df[(weather_df['Max Temp']>=70)
                             &(weather_df['Max Temp']<=85)
                             &(weather_df['Humidity']<=70)
                             &(weather_df['Cloudiness']==0)]
ideal_locations

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,Salalah,17.0151,54.0924,73.40,38,0,9.22,OM,1613005175
61,Hobart,-42.8794,147.3294,73.00,46,0,5.75,AU,1613005081
95,Oyotún,-6.8519,-79.3036,78.80,61,0,16.11,PE,1613005178
136,Faanui,-16.4833,-151.7500,82.87,67,0,14.85,PF,1613005183
145,Geraldton,-28.7667,114.6000,71.60,46,0,19.57,AU,1613005184
146,Marabba,12.3500,32.1833,72.97,21,0,9.17,SD,1613005184
157,Filingué,14.3521,3.3168,73.78,17,0,8.12,NE,1613005185
158,Saint-Louis,16.3333,-15.0000,77.00,27,0,2.30,SN,1613005185
206,Eten,-6.9081,-79.8642,78.80,61,0,16.11,PE,1613005190
298,Djibo,14.1022,-1.6306,73.98,17,0,7.70,BF,1613005198


### Hotel Map

In [6]:
hotel_df = ideal_locations.copy()

In [7]:
hotel_df['Hotel Name'] = ''

In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    'radius': '5000',
    'type' : 'lodging',
    'key' : g_key    
}

print(f'Initialising search for hotels in your ideal locations:\n')

for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    
    city_name = row['City']
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
        
    try:
        print(f"Found a hotel option for your vacation: {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

print('\nSearch Complete')

hotel_df.dropna(inplace = True)

Initialising search for hotels in your ideal locations:

Retrieving Results for Index 58: Salalah.
Found a hotel option for your vacation: HAMDAN PLAZA HOTEL SALALAH.
------------
Retrieving Results for Index 61: Hobart.
Found a hotel option for your vacation: St Ives Hobart Accommodation Tasmania.
------------
Retrieving Results for Index 95: Oyotún.
Found a hotel option for your vacation: Cerro La Compuerta.
------------
Retrieving Results for Index 136: Faanui.
Found a hotel option for your vacation: Bora-Bora Pearl Beach Resort & Spa.
------------
Retrieving Results for Index 145: Geraldton.
Found a hotel option for your vacation: Broadwater Mariner Resort.
------------
Retrieving Results for Index 146: Marabba.
Missing field/result... skipping.
------------
Retrieving Results for Index 157: Filingué.
Missing field/result... skipping.
------------
Retrieving Results for Index 158: Saint-Louis.
Missing field/result... skipping.
------------
Retrieving Results for Index 206: Eten.
Fo

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

In [10]:
# Add marker layer ontop of heat map
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
city_names = hotel_df['City'].tolist()

hotel_markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(hotel_markers)

# Display figure

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…